#  YouTube 자막 추출 + Gemini 요약 + Notion 연동 (통합)

**YouTube 영상 자막을 추출하고, Gemini AI로 요약한 후, Notion에 자동으로 저장하는 완전 자동화 시스템**

###  통합된 기능:
-  YouTube 자막 추출 (최신 API 사용)
-  Google Gemini Pro AI 요약
-  Notion 페이지 자동 생성 및 저장
-  사용자 정의 요약 스타일 지원

In [ ]:
# 패키지 import
import asyncio
import json
import logging
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Union, Any 
from urllib.parse import urlparse
import re
from abc import ABC, abstractmethod

# 웹 크롤링 및 HTTP
import requests
from bs4 import BeautifulSoup
import validators

# Google GenAI
import google.generativeai as genai
from google.genai import types

# Notion API
from notion_client import Client
import numpy as np
import pandas as pd

# 유튜브 (최신 API)
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
# YouTube Data API 추가
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 환경변수
from dotenv import load_dotenv
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 모든 패키지가 성공적으로 로드되었습니다!")

##  API 키 및 환경 설정

In [ ]:
# 환경 변수 설정
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
PARENT_PAGE_ID = os.getenv("NOTION_PARENT_PAGE") # 상위 페이지 ID
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # Gemini API 키
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY") # Youtube API 키


print("🔑 환경 변수 상태:")
print(f"  • NOTION_TOKEN: {'✅ 설정됨' if NOTION_TOKEN else '❌ 설정 필요'}")
print(f"  • PARENT_PAGE_ID: {'✅ 설정됨' if PARENT_PAGE_ID else '❌ 설정 필요'}")
print(f"  • NOTION_DATABASE_ID: {'✅ 설정됨' if NOTION_DATABASE_ID else '❌ 설정 필요'}")
print(f"  • GEMINI_API_KEY: {'✅ 설정됨' if GEMINI_API_KEY else '❌ 설정 필요'}")

if not all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY, YOUTUBE_API_KEY]):
    print("⚠️ 필요한 환경 변수가 설정되지 않았습니다!")
    print("💡 .env 파일에 다음 항목들을 설정해주세요:")
    print("   - NOTION_TOKEN=your_notion_token")
    print("   - notion_parent_page=your_parent_page_id")
    print("   - GOOGLE_AI_API_KEY=your_google_ai_key")
    print("   - YOUTUBE_API_KEY=your_youtube_data_api_key")

## BaseTool(모든 도구들의 기본 인터페이스)

In [ ]:
# Tool 기본 인터페이스
class BaseTool(ABC):
    """모든 도구의 기본 인터페이스"""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.logger = logging.getLogger(f"Tool.{name}")
    
    @abstractmethod
    async def execute(self, **kwargs) -> Dict[str, Any]:
        """도구 실행 (추상 메서드)"""
        pass
    
    def log_info(self, message: str):
        """로깅 헬퍼"""
        self.logger.info(f"[{self.name}] {message}")
        print(f"🔧 [{self.name}] {message}")
    
    def log_error(self, message: str):
        """에러 로깅 헬퍼"""
        self.logger.error(f"[{self.name}] {message}")
        print(f"❌ [{self.name}] {message}")

print("📐 BaseTool 인터페이스 정의 완료!")

## BaseAgent

In [ ]:
# Agent 기본 인터페이스
class BaseAgent(ABC):
    """모든 에이전트의 기본 인터페이스"""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.logger = logging.getLogger(f"Agent.{name}")
    
    @abstractmethod
    async def execute(self, **kwargs) -> Dict[str, Any]:
        """에이전트 실행 (추상 메서드)"""
        pass
    
    def log_info(self, message: str):
        """로깅 헬퍼"""
        self.logger.info(f"[{self.name}] {message}")
        print(f"🤖 [{self.name}] {message}")
    
    def log_error(self, message: str):
        """에러 로깅 헬퍼"""
        self.logger.error(f"[{self.name}] {message}")
        print(f"❌ [{self.name}] {message}")

print("🤖 BaseAgent 인터페이스 정의 완료!")

## YouTubeSearchTool(YouTube 검색 도구)

In [ ]:
class YouTubeSearchTool(BaseTool):
    """YouTube 검색 도구 - 검색어로 영상 검색"""
    
    def __init__(self, api_key: str):
        super().__init__("YouTubeSearch", "YouTube에서 검색어로 영상을 검색하는 도구")
        self.api_key = api_key
        self.youtube_service = None
        
    def _get_youtube_service(self):
        if self.youtube_service is None:
            self.youtube_service = build('youtube', 'v3', developerKey=self.api_key)
        return self.youtube_service
    
    async def execute(self, query: str, max_results: int = 3, **kwargs) -> Dict[str, Any]:
        try:
            self.log_info(f"검색 시작: '{query}' (최대 {max_results}개)")
            
            youtube = self._get_youtube_service()
            
            search_response = youtube.search().list(
                q=query,
                part='id,snippet',
                maxResults=max_results,
                type='video',
                order='relevance',
                regionCode='KR',
                relevanceLanguage='ko'
            ).execute()
            
            videos = []
            for item in search_response['items']:
                video_id = item['id']['videoId']
                snippet = item['snippet']
                
                video_info = {
                    'video_id': video_id,
                    'title': snippet['title'],
                    'description': snippet['description'],
                    'channel_title': snippet['channelTitle'],
                    'published_at': snippet['publishedAt'],
                    'thumbnail_url': snippet['thumbnails']['high']['url'],
                    'url': f'https://www.youtube.com/watch?v={video_id}'
                }
                videos.append(video_info)
            
            self.log_info(f"검색 완료: {len(videos)}개 영상 찾음")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'query': query,
                    'videos': videos,
                    'total_count': len(videos)
                }
            }
            
        except HttpError as e:
            error_msg = f"YouTube API 오류: {str(e)}"
            self.log_error(error_msg)
            return {'success': False, 'error': error_msg, 'data': None}
        except Exception as e:
            error_msg = f"검색 실패: {str(e)}"
            self.log_error(error_msg)
            return {'success': False, 'error': error_msg, 'data': None}

## YouTubeTranscriptTool(YouTube 자막 추출 도구 (다국어 지원))

In [ ]:
class YouTubeTranscriptTool(BaseTool):
    """YouTube 자막 추출 도구 (다국어 지원)"""
    
    def __init__(self):
        super().__init__("YouTubeTranscript", "YouTube 영상에서 자막을 추출하는 도구")
    
    def extract_video_id(self, url: str) -> str:
        """YouTube URL에서 비디오 ID 추출"""
        patterns = [
            r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([^&\n?#]+)',
            r'youtube\.com\/embed\/([^&\n?#]+)',
            r'youtube\.com\/v\/([^&\n?#]+)',
            r'youtube\.com\/shorts\/([^&\n?#]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None
    
    async def execute(self, url: str, **kwargs) -> Dict[str, Any]:
        """YouTube 자막 추출 (최신 API 사용)"""
        try:
            self.log_info(f"자막 추출 시작: {url}")
            
            video_id = self.extract_video_id(url)
            if not video_id:
                return {
                    'success': False, 
                    'error': '유효하지 않은 YouTube URL입니다.',
                    'data': None
                }
            
            self.log_info(f"비디오 ID: {video_id}")
            
            # 최신 API 사용
            api = YouTubeTranscriptApi()
            
            # 다양한 언어로 시도
            languages_to_try = [['ko'], ['en'], ['a.ko'], ['a.en']]
            
            for languages in languages_to_try:
                try:
                    self.log_info(f"{languages[0]} 언어 자막 시도 중...")
                    fetched_transcript = api.fetch(video_id, languages=languages)
                    
                    # FetchedTranscriptSnippet 객체의 속성 접근
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    self.log_info(f"자막 추출 성공! (언어: {languages[0]}, 길이: {len(content)}자)")
                    return {
                        'success': True,
                        'error': None,
                        'data': {
                            'content': content,
                            'language': languages[0],
                            'video_id': video_id,
                            'length': len(content),
                            'url': url
                        }
                    }
                except Exception as e:
                    self.log_info(f"{languages[0]} 실패: {str(e)[:50]}...")
                    continue
            
            # 사용 가능한 자막 목록 확인
            try:
                self.log_info("사용 가능한 자막 목록 확인 중...")
                transcript_list = api.list(video_id)
                
                if len(transcript_list) > 0:
                    first_transcript = transcript_list[0]
                    self.log_info(f"첫 번째 자막 언어로 시도: {first_transcript.language_code}")
                    
                    fetched_transcript = first_transcript.fetch()
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    self.log_info(f"자막 추출 성공! (언어: {first_transcript.language_code}, 길이: {len(content)}자)")
                    return {
                        'success': True,
                        'error': None,
                        'data': {
                            'content': content,
                            'language': first_transcript.language_code,
                            'video_id': video_id,
                            'length': len(content),
                            'url': url
                        }
                    }
                
            except Exception as e:
                self.log_error(f"자막 목록 확인 실패: {str(e)}")
            
            return {
                'success': False, 
                'error': '자막을 찾을 수 없거나 네트워크 제한이 있습니다.',
                'data': None
            }
            
        except Exception as e:
            self.log_error(f"자막 추출 중 오류: {str(e)}")
            return {
                'success': False, 
                'error': f'오류 발생: {str(e)}',
                'data': None
            }

## TranslationTool(Gemini AI 번역 도구)

In [ ]:
class TranslationTool(BaseTool):
    """Gemini AI 번역 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("Translation", "Gemini AI로 텍스트를 번역하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 번역 모델 초기화 완료")
    
    async def execute(self, content: str, source_lang: str, target_lang: str = 'ko', 
                    **kwargs) -> Dict[str, Any]:
        """고품질 번역 실행"""
        try:
            self.log_info(f"번역 시작: {source_lang} → {target_lang} (길이: {len(content)}자)")
            
            # 번역 프롬프트
            translation_prompt = f"""
다음은 YouTube 영상의 {source_lang} 자막입니다. 이를 자연스럽고 정확한 {target_lang}로 번역해주세요.

## 번역 지침:
1. **의미 보존**: 원문의 의미를 정확히 전달
2. **자연스러운 번역**: 직역이 아닌 의역으로 자연스럽게
3. **전문용어 유지**: 기술용어나 고유명사는 적절히 보존
4. **맥락 고려**: 영상의 맥락을 고려하여 번역
5. **구어체 유지**: 영상의 톤앤매너 유지

번역할 내용:
{content}

위 내용을 {target_lang}로 번역해주세요. 번역된 내용만 출력하고, 다른 설명은 추가하지 마세요.
"""
            
            # Gemini 번역 요청
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    translation_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 번역을 생성하지 못했습니다.',
                    'data': None
                }
            
            translated_content = response.text.strip()
            
            self.log_info(f"번역 완료 (원본: {len(content)}자 → 번역: {len(translated_content)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'translated_content': translated_content,
                    'original_content': content,
                    'source_lang': source_lang,
                    'target_lang': target_lang,
                    'original_length': len(content),
                    'translated_length': len(translated_content)
                }
            }
            
        except Exception as e:
            self.log_error(f"번역 실패: {str(e)}")
            return {
                'success': False,
                'error': f'번역 실패: {str(e)}',
                'data': None
            }

print("🌍 TranslationTool 정의 완료!")

## GeminiSummaryTool(Gemini AI 요약 도구)

In [ ]:
class GeminiSummaryTool(BaseTool):
    """Gemini AI 요약 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("GeminiSummary", "Gemini AI로 텍스트를 요약하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 모델 초기화 완료")
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """요약 실행"""
        try:
            self.log_info(f"Gemini 요약 시작 (콘텐츠 길이: {len(content)}자)")
            
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
2. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
3. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
4. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 비디오 정보 추가
            video_context = ""
            if video_info:
                video_context = f"""
## 📺 영상 정보:
- 제목: {video_info.get('title', 'Unknown')}
- 채널: {video_info.get('channel', 'Unknown')}
- 비디오 ID: {video_info.get('video_id', 'Unknown')}
"""
            
            # 콘텐츠 길이 제한
            max_content_length = 25000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                self.log_info(f"콘텐츠가 길어서 {max_content_length}자로 축약")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}{video_context}\n\n## 원본 콘텐츠:\n{content}"
            
            # Gemini 호출
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.',
                    'data': None
                }
            
            summary = response.text.strip()
            
            self.log_info(f"요약 생성 완료 (길이: {len(summary)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'summary': summary,
                    'original_length': len(content),
                    'summary_length': len(summary),
                    'tokens_used': len(full_prompt.split()) + len(summary.split())
                }
            }
            
        except Exception as e:
            self.log_error(f"요약 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Gemini 요약 실패: {str(e)}',
                'data': None
            }


## YouTubeContentAgent (콘텐츠 요약 전문 에이전트)

In [ ]:
class ContentSummaryAgent(BaseAgent):
    """GeminiSummaryTool을 활용한 콘텐츠 요약 전문 에이전트"""
    
    def __init__(self, summary_tool: GeminiSummaryTool):
        super().__init__("ContentSummary", "GeminiSummaryTool을 활용한 콘텐츠 요약 전문 에이전트", [summary_tool])
        self.summary_tool = summary_tool
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """콘텐츠 요약 실행"""
        try:
            self.log_info(f"콘텐츠 요약 시작 (길이: {len(content)}자)")
            
            # Gemini 요약 도구 사용
            summary_result = await self.summary_tool.execute(
                content=content,
                custom_prompt=custom_prompt,
                video_info=video_info
            )
            
            return summary_result
            
        except Exception as e:
            self.log_error(f"콘텐츠 요약 실패: {str(e)}")
            return {
                'success': False,
                'error': f'콘텐츠 요약 실패: {str(e)}',
                'data': None
            }

print("📝 ContentSummaryAgent 정의 완료!")

## GeminiSummaryTool(Gemini AI 요약 도구)

In [ ]:
class GeminiSummaryTool(BaseTool):
    """Gemini API를 호출하여 텍스트를 요약하는 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("GeminiSummary", "Gemini API를 호출하여 텍스트를 요약하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 모델 초기화 완료")
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """요약 실행"""
        try:
            self.log_info(f"Gemini 요약 시작 (콘텐츠 길이: {len(content)}자)")
            
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
2. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
3. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
4. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 비디오 정보 추가
            video_context = ""
            if video_info:
                video_context = f"""
## 📺 영상 정보:
- 제목: {video_info.get('title', 'Unknown')}
- 채널: {video_info.get('channel', 'Unknown')}
- 비디오 ID: {video_info.get('video_id', 'Unknown')}
"""
            
            # 콘텐츠 길이 제한
            max_content_length = 25000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                self.log_info(f"콘텐츠가 길어서 {max_content_length}자로 축약")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}{video_context}\n\n## 원본 콘텐츠:\n{content}"
            
            # Gemini 호출
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.',
                    'data': None
                }
            
            summary = response.text.strip()
            
            self.log_info(f"요약 생성 완료 (길이: {len(summary)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'summary': summary,
                    'original_length': len(content),
                    'summary_length': len(summary),
                    'compression_ratio': round(len(summary) / len(content), 2),
                    'tokens_used': len(full_prompt.split()) + len(summary.split())
                }
            }
            
        except Exception as e:
            self.log_error(f"요약 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Gemini 요약 실패: {str(e)}',
                'data': None
            }

## ContentSummaryAgent(콘텐츠 요약 전문 에이전트)

In [ ]:
class ContentSummaryAgent(BaseAgent):
    """콘텐츠 요약 전문 에이전트"""
    
    def __init__(self, summary_tool: GeminiSummaryTool):
        super().__init__("ContentSummary", "텍스트 콘텐츠 요약 처리", [summary_tool])
        self.summary_tool = summary_tool
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """콘텐츠 요약 실행"""
        try:
            self.log_info(f"콘텐츠 요약 시작 (길이: {len(content)}자)")
            
            # Gemini 요약 도구 사용
            summary_result = await self.summary_tool.execute(
                content=content,
                custom_prompt=custom_prompt,
                video_info=video_info
            )
            
            return summary_result
            
        except Exception as e:
            self.log_error(f"콘텐츠 요약 실패: {str(e)}")
            return {
                'success': False,
                'error': f'콘텐츠 요약 실패: {str(e)}',
                'data': None
            }

print("📝 GeminiSummaryTool + ContentSummaryAgent 정의 완료!")

## Notion Agent(페이지 생성 도구)

In [ ]:
class NotionPageTool(BaseTool):
    """Notion 페이지 생성 도구"""
    
    def __init__(self, notion_token: str, parent_page_id: str):
        super().__init__("NotionPage", "Notion에 페이지를 생성하는 도구")
        self.notion_token = notion_token
        self.parent_page_id = parent_page_id
        
        self.log_info("Notion API 클라이언트 초기화 완료")
    
    async def execute(self, title: str, summary: str, original_content: str, 
                    url: str, metadata: Dict = None, **kwargs) -> Dict[str, Any]:
        """Notion 페이지 생성 실행"""
        try:
            self.log_info(f"Notion 페이지 생성 시작: {title}")
            
            # Notion API 헤더
            headers = {
                "Authorization": f"Bearer {self.notion_token}",
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
            
            # 현재 시간
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 제목이 없으면 자동 생성
            if not title:
                title = f"YouTube 요약 - {current_time}"
            
            # 새 페이지 생성
            new_page = {
                "parent": {"page_id": self.parent_page_id},
                "properties": {
                    "title": [{"text": {"content": title}}]
                }
            }
            
            # 페이지 생성 요청
            page_response = requests.post(
                "https://api.notion.com/v1/pages", 
                headers=headers, 
                data=json.dumps(new_page)
            )
            
            if page_response.status_code != 200:
                return {
                    'success': False,
                    'error': f'페이지 생성 실패: {page_response.text}',
                    'data': None
                }
            
            page_data = page_response.json()
            page_id = page_data['id']
            page_url = page_data['url']
            
            self.log_info(f"페이지 생성 완료: {page_id}")
            
            # 콘텐츠 블록 생성
            blocks = self.create_content_blocks(summary, original_content, url, metadata)
            
            # 페이지에 블록 추가
            blocks_response = requests.patch(
                f"https://api.notion.com/v1/blocks/{page_id}/children",
                headers=headers,
                data=json.dumps({"children": blocks})
            )
            
            if blocks_response.status_code != 200:
                self.log_error(f"콘텐츠 추가 실패: {blocks_response.text}")
            else:
                self.log_info("페이지 콘텐츠 추가 완료")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'page_id': page_id,
                    'page_url': page_url,
                    'title': title
                }
            }
            
        except Exception as e:
            self.log_error(f"페이지 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Notion 페이지 생성 실패: {str(e)}',
                'data': None
            }
    
    def create_content_blocks(self, summary: str, original_content: str, 
                            url: str, metadata: Dict = None) -> List[Dict]:
        """Notion 페이지 콘텐츠 블록 생성 (기존 로직 그대로)"""
        blocks = []
        
        # 현재 시간
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 1. YouTube URL 링크
        blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {"type": "text", "text": {"content": "🔗 원본 영상: "}},
                    {"type": "text", "text": {"content": url, "link": {"url": url}}}
                ]
            }
        })
        
        # 2. 메타데이터
        if metadata:
            meta_parts = []
            meta_parts.append(f"📊 언어: {metadata.get('language', 'unknown')}")
            meta_parts.append(f"길이: {metadata.get('length', 0):,}자")
            
            if metadata.get('translated'):
                meta_parts.append(f"번역: {metadata.get('original_language')} → 한국어")
            
            meta_parts.append(f"생성일: {current_time}")
            meta_text = " | ".join(meta_parts)
            
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": meta_text}}]
                }
            })
        
        # 3. 구분선
        blocks.append({"object": "block", "type": "divider", "divider": {}})
        
        # 4. AI 요약 제목
        blocks.append({
            "object": "block",
            "type": "heading_1",
            "heading_1": {
                "rich_text": [{"type": "text", "text": {"content": "🤖 Gemini AI 요약"}}]
            }
        })
        
        # 5. 요약 내용 (2000자씩 나누어 추가)
        summary_chunks = [summary[i:i+2000] for i in range(0, len(summary), 2000)]
        for chunk in summary_chunks:
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": chunk}}]
                }
            })
        
        # 6. 구분선
        blocks.append({"object": "block", "type": "divider", "divider": {}})
        
        # 7. 원문 자막 (토글 블록으로)
        original_chunks = [original_content[i:i+2000] for i in range(0, len(original_content), 2000)]
        original_blocks = []
        
        for chunk in original_chunks:
            original_blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": chunk}}]
                }
            })
        
        blocks.append({
            "object": "block",
            "type": "toggle",
            "toggle": {
                "rich_text": [{"type": "text", "text": {"content": "📄 원문 자막 보기"}}],
                "children": original_blocks[:50]  # 최대 50개 블록
            }
        })
        
        return blocks

## NotionPublishAgent(발행 전문 에이전트)

In [ ]:
class NotionPublishAgent(BaseAgent):
    """Notion 페이지 발행 전문 에이전트"""
    
    def __init__(self, notion_tool: NotionPageTool):
        super().__init__("NotionPublish", "Notion 페이지 생성 및 발행")
        self.notion_tool = notion_tool
    
    async def execute(self, 
                     video_info: Dict,
                     transcript_data: Dict,
                     summary_data: Dict,
                     custom_title: str = None,
                     **kwargs) -> Dict[str, Any]:
        """Notion 페이지 발행 실행"""
        try:
            self.log_info("Notion 페이지 발행 시작")
            
            # positional arguments로 전달
            title = custom_title or video_info.get('title', '제목 없음')
            summary = summary_data['summary']
            original_content = transcript_data['content']
            url = video_info.get('url', '')
            
            result = await self.notion_tool.execute(
                title,           # 첫 번째 positional argument
                summary,         # 두 번째 positional argument
                original_content, # 세 번째 positional argument
                url              # 네 번째 positional argument
            )
            
            if result['success']:
                self.log_info("Notion 페이지 발행 완료")
                return result
            else:
                self.log_error(f"페이지 발행 실패: {result['error']}")
                return result
                
        except Exception as e:
            error_msg = f"Notion 페이지 발행 실패: {str(e)}"
            self.log_error(error_msg)
            return {'success': False, 'error': error_msg, 'data': None}

YouTubeContentAgent

In [ ]:
class YouTubeContentAgent(BaseAgent):
    """YouTube 콘텐츠 처리 전문 에이전트"""
    
    def __init__(self, transcript_tool: YouTubeTranscriptTool, translation_tool=None):
        super().__init__("YouTubeContent", "YouTube 자막 추출 및 번역 처리")
        self.transcript_tool = transcript_tool
        self.translation_tool = translation_tool
    
    async def execute(self, url: str = None, video_id: str = None, **kwargs) -> Dict[str, Any]:
        """YouTube 콘텐츠 처리 실행"""
        try:
            self.log_info("YouTube 콘텐츠 처리 시작")
            
            # 자막 추출
            result = await self.transcript_tool.execute(url=url, video_id=video_id)
            
            if result['success']:
                self.log_info(f"자막 추출 완료: {result['data']['language']}")
                return result
            else:
                self.log_error(f"자막 추출 실패: {result['error']}")
                return result
                
        except Exception as e:
            error_msg = f"YouTube 콘텐츠 처리 실패: {str(e)}"
            self.log_error(error_msg)
            return {'success': False, 'error': error_msg, 'data': None}


class ContentSummaryAgent(BaseAgent):
    """GeminiSummaryTool을 활용한 콘텐츠 요약 전문 에이전트"""
    
    def __init__(self, summary_tool: GeminiSummaryTool):
        super().__init__("ContentSummary", "AI 기반 콘텐츠 요약")
        self.summary_tool = summary_tool
    
    async def execute(self, content: str, custom_prompt: str = None, **kwargs) -> Dict[str, Any]:
        """콘텐츠 요약 실행"""
        try:
            self.log_info("콘텐츠 요약 시작")
            
            result = await self.summary_tool.execute(content=content, custom_prompt=custom_prompt)
            
            if result['success']:
                self.log_info("요약 완료")
                return result
            else:
                self.log_error(f"요약 실패: {result['error']}")
                return result
                
        except Exception as e:
            error_msg = f"콘텐츠 요약 실패: {str(e)}"
            self.log_error(error_msg)
            return {'success': False, 'error': error_msg, 'data': None}
print("🤖 Agent 클래스들 정의 완료!")

## YouTube-Gemini-Notion (워크플로우 코디네이터)

In [ ]:
class YouTubeGeminiNotionCoordinator:
    """YouTube → Gemini → Notion 전체 워크플로우 조율 코디네이터"""
    
    def __init__(self,
                 youtube_agent: YouTubeContentAgent,
                 summary_agent: ContentSummaryAgent,
                 notion_agent: NotionPublishAgent):
        
        self.youtube_agent = youtube_agent
        self.summary_agent = summary_agent
        self.notion_agent = notion_agent
        
        self.logger = logging.getLogger("Coordinator")
        
        print("🎯 YouTubeGeminiNotionCoordinator 초기화 완료!")
        print(f"  • YouTube Agent: {youtube_agent.name}")
        print(f"  • Summary Agent: {summary_agent.name}")
        print(f"  • Notion Agent: {notion_agent.name}")
    
    def log_info(self, message: str):
        """로깅 헬퍼"""
        self.logger.info(f"[Coordinator] {message}")
        print(f"🎯 [Coordinator] {message}")
    
    def log_error(self, message: str):
        """에러 로깅 헬퍼"""
        self.logger.error(f"[Coordinator] {message}")
        print(f"❌ [Coordinator] {message}")
    
    async def process_single_video(self, 
                                 url: str = None,
                                 video_id: str = None,
                                 video_info: Dict = None,
                                 custom_prompt: str = None, 
                                 custom_title: str = None) -> Dict[str, Any]:
        """단일 영상 처리 워크플로우"""
        
        self.log_info("🚀 단일 영상 처리 워크플로우 시작")
        print(f"📺 처리 대상: {url or f'Video ID: {video_id}'}")
        
        errors = []
        
        try:
            # 1단계: YouTube 콘텐츠 처리 (자막 추출)
            self.log_info("1️⃣ YouTube 콘텐츠 처리 시작...")
            
            youtube_result = await self.youtube_agent.execute(
                url=url,
                video_id=video_id
            )
            
            if not youtube_result['success']:
                error_msg = f"YouTube 처리 실패: {youtube_result['error']}"
                self.log_error(error_msg)
                errors.append(error_msg)
                return {
                    'overall_success': False,
                    'errors': errors,
                    'final_result': None
                }
            
            transcript_data = youtube_result['data']
            self.log_info(f"✅ 자막 추출 완료 (언어: {transcript_data['language']}, 길이: {transcript_data['length']:,}자)")
            
            # 2단계: AI 요약 처리
            self.log_info("2️⃣ AI 요약 처리 시작...")
            
            summary_result = await self.summary_agent.execute(
                content=transcript_data['content'],
                custom_prompt=custom_prompt
            )
            
            if not summary_result['success']:
                error_msg = f"요약 처리 실패: {summary_result['error']}"
                self.log_error(error_msg)
                errors.append(error_msg)
                return {
                    'overall_success': False,
                    'errors': errors,
                    'final_result': None
                }
            
            summary_data = summary_result['data']
            self.log_info(f"✅ 요약 완료 (압축률: {summary_data['compression_ratio']*100:.1f}%)")
            
            # 3단계: Notion 페이지 생성
            self.log_info("3️⃣ Notion 페이지 생성 시작...")
            
            # video_info가 없으면 기본값 생성
            if not video_info:
                video_info = {
                    'url': url or f'https://www.youtube.com/watch?v={video_id}',
                    'title': '처리된 YouTube 영상',
                    'channel_title': '알 수 없음'
                }
            
            notion_result = await self.notion_agent.execute(
                video_info=video_info,
                transcript_data=transcript_data,
                summary_data=summary_data,
                custom_title=custom_title
            )
            
            if not notion_result['success']:
                error_msg = f"Notion 페이지 생성 실패: {notion_result['error']}"
                self.log_error(error_msg)
                errors.append(error_msg)
                return {
                    'overall_success': False,
                    'errors': errors,
                    'final_result': None
                }
            
            notion_data = notion_result['data']
            self.log_info(f"✅ Notion 페이지 생성 완료!")
            
            # 최종 결과
            self.log_info("🎉 전체 워크플로우 완료!")
            
            final_result = {
                'youtube_data': transcript_data,
                'summary_data': summary_data,
                'notion_data': notion_data,
                'page_url': notion_data['page_url']
            }
            
            return {
                'overall_success': True,
                'errors': [],
                'final_result': final_result
            }
            
        except Exception as e:
            error_msg = f"코디네이터 워크플로우 중 오류: {str(e)}"
            self.log_error(error_msg)
            errors.append(error_msg)
            
            return {
                'overall_success': False,
                'errors': errors,
                'final_result': None
            }
    
    async def process_search_results(self,
                                   search_results: List[Dict],
                                   custom_prompt: str = None,
                                   custom_title: str = None,
                                   process_all: bool = False) -> Dict[str, Any]:
        """검색 결과 영상들 처리"""
        
        self.log_info(f"🔍 검색 결과 처리 시작 ({len(search_results)}개 영상)")
        
        if not search_results:
            return {
                'overall_success': False,
                'errors': ['검색 결과가 없습니다.'],
                'final_result': None
            }
        
        # 첫 번째 영상만 처리하거나 모든 영상 처리
        videos_to_process = search_results if process_all else [search_results[0]]
        
        results = []
        errors = []
        
        for i, video_info in enumerate(videos_to_process):
            self.log_info(f"📺 영상 {i+1}/{len(videos_to_process)} 처리 중: {video_info['title'][:50]}...")
            
            result = await self.process_single_video(
                url=video_info['url'],
                video_info=video_info,
                custom_prompt=custom_prompt,
                custom_title=custom_title
            )
            
            if result['overall_success']:
                results.append(result['final_result'])
                self.log_info(f"✅ 영상 {i+1} 처리 완료!")
            else:
                errors.extend(result['errors'])
                self.log_error(f"❌ 영상 {i+1} 처리 실패")
        
        if results:
            return {
                'overall_success': True,
                'errors': errors,
                'final_result': results[0] if len(results) == 1 else results,
                'processed_count': len(results)
            }
        else:
            return {
                'overall_success': False,
                'errors': errors,
                'final_result': None,
                'processed_count': 0
            }

print("🎯 YouTubeGeminiNotionCoordinator 정의 완료!")

## 통합 Agent 초기화

In [ ]:
# 시스템 팩토리 - 전체 시스템을 쉽게 구성
class YouTubeGeminiNotionFactory:
    """YouTube-Gemini-Notion Agent 협업 시스템 팩토리"""
    
    @staticmethod
    def create_system(notion_token: str,
                     parent_page_id: str,
                     gemini_api_key: str,
                     gemini_model: str = 'gemini-2.5-flash-lite') -> YouTubeGeminiNotionCoordinator:
        """
        Agent 협업 시스템 생성
        
        Args:
            notion_token: Notion API 토큰
            parent_page_id: Notion 상위 페이지 ID
            gemini_api_key: Gemini API 키
            gemini_model: Gemini 모델명
            
        Returns:
            YouTubeGeminiNotionCoordinator: 설정된 코디네이터
        """
        
        print("🏗️ Agent 협업 시스템 구축 중...")
        print("📊 아키텍처: Tool → Agent → Coordinator 3계층")
        
        # 1. Tool 계층 생성
        print("\n🔧 Tool 계층 초기화 중...")
        
        # YouTube 도구들
        transcript_tool = YouTubeTranscriptTool()
        
        # AI 도구들 (번역 포함)
        summary_tool = GeminiSummaryTool(gemini_api_key, gemini_model)
        translation_tool = TranslationTool(gemini_api_key, gemini_model)
        
        # Notion 도구
        notion_tool = NotionPageTool(notion_token, parent_page_id)
        
        print(f"  ✅ YouTubeTranscriptTool: 다국어 자막 추출")
        print(f"  ✅ GeminiSummaryTool: AI 요약 ({gemini_model})")
        print(f"  ✅ TranslationTool: 자동 번역 ({gemini_model})")
        print(f"  ✅ NotionPageTool: 페이지 생성")

        # 2. Agent 계층 생성
        print("\n🤖 Agent 계층 초기화 중...")
        
        youtube_agent = YouTubeContentAgent(
            transcript_tool=transcript_tool,
            translation_tool=translation_tool  # 번역 기능 포함
        )
        
        summary_agent = ContentSummaryAgent(summary_tool)
        notion_agent = NotionPublishAgent(notion_tool)
        
        print(f"  ✅ YouTubeContentAgent: 자막 + 번역 처리")
        print(f"  ✅ ContentSummaryAgent: AI 요약 전문")
        print(f"  ✅ NotionPublishAgent: 페이지 발행 전문")

        # 3. Coordinator 계층 생성
        print("\n🎯 Coordinator 계층 초기화 중...")
        
        coordinator = YouTubeGeminiNotionCoordinator(
            youtube_agent=youtube_agent,
            summary_agent=summary_agent,
            notion_agent=notion_agent
        )
        
        print("\n🎉 Agent 협업 시스템 구축 완료!")
        print("📋 시스템 특징:")
        print("  • 🔧 모듈화: 독립적인 Tool들")
        print("  • 🤖 전문화: 각 영역별 Agent들")
        print("  • 🎯 조율: 체계적인 Coordinator")
        print("  • 🌍 다국어: 자동 번역 지원")
        print("  • 🛠️ 확장성: 새로운 Tool/Agent 쉽게 추가")
        
        return coordinator
    @staticmethod
    def from_env_vars() -> YouTubeGeminiNotionCoordinator:
        """환경 변수에서 Agent 협업 시스템 생성"""
        # 환경 변수 로드
        NOTION_TOKEN = os.getenv("NOTION_TOKEN")
        PARENT_PAGE_ID = os.getenv("NOTION_PARENT_PAGE") or os.getenv("notion_parent_page")
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_AI_API_KEY")
        
        # 필수 변수 확인
        required_vars = {
            'NOTION_TOKEN': NOTION_TOKEN,
            'PARENT_PAGE_ID': PARENT_PAGE_ID,
            'GEMINI_API_KEY': GEMINI_API_KEY
        }
        
        missing_vars = [name for name, value in required_vars.items() if not value]
        
        if missing_vars:
            raise ValueError(f"필수 환경 변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
        
        print("🔑 환경 변수에서 Agent 협업 시스템 생성 중...")
        
        return YouTubeGeminiNotionFactory.create_system(
            notion_token=NOTION_TOKEN,
            parent_page_id=PARENT_PAGE_ID,
            gemini_api_key=GEMINI_API_KEY
        )

print("🏭 YouTubeGeminiNotionFactory 정의 완료!")

## 자동화 실행

In [ ]:
# Agent 협업 시스템 초기화 및 실행

# 시스템 초기화
if not all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY, YOUTUBE_API_KEY]):
    print("❌ 필요한 환경변수가 설정되지 않았습니다!")
    print("💡 .env 파일에 다음을 설정해주세요:")
    print("   - NOTION_TOKEN, NOTION_PARENT_PAGE, GEMINI_API_KEY, YOUTUBE_API_KEY")
else:
    print("✅ 모든 환경변수가 설정되었습니다!")
    
    print("\n🎯 어떤 방식으로 YouTube 영상을 처리하시겠습니까?")
    print("1️⃣ 검색어 입력 (예: 파이썬, 자바, 머신러닝 등)")
    print("2️⃣ YouTube URL 직접 입력")
    
    choice = input("\n선택 (1 또는 2): ").strip()
    
    # 도구 초기화 (들여쓰기 수정)
    search_tool = YouTubeSearchTool(YOUTUBE_API_KEY)
    transcript_tool = YouTubeTranscriptTool()
    summary_tool = GeminiSummaryTool(GEMINI_API_KEY)
    notion_tool = NotionPageTool(NOTION_TOKEN, PARENT_PAGE_ID)
    
    # Agent 초기화
    youtube_agent = YouTubeContentAgent(transcript_tool)
    summary_agent = ContentSummaryAgent(summary_tool)
    notion_agent = NotionPublishAgent(notion_tool)
    
    # Coordinator 초기화
    coordinator = YouTubeGeminiNotionCoordinator(youtube_agent, summary_agent, notion_agent)
    
    if choice == "1":
        # 검색어 방식
        search_query = input("🔍 검색어를 입력하세요: ").strip()
        if not search_query:
            search_query = "파이썬 기초"
        
        # 검색 실행
        search_result = await search_tool.execute(search_query)
        if search_result['success']:
            videos = search_result['data']['videos']
            print(f"\n📋 검색 결과:")
            for i, video in enumerate(videos):
                print(f"  {i+1}. {video['title'][:60]}...")
                print(f"     채널: {video['channel_title']}")
            
            # 사용자가 처리 방식 선택
            print(f"\n🎯 어떻게 처리하시겠습니까?")
            print("1️⃣ 첫 번째 영상만 처리")
            print("2️⃣ 모든 영상 처리 (3개)")
            print("3️⃣ 특정 영상 선택")

            process_choice = input("선택 (1, 2, 또는 3): ").strip()
                
            # 공통 옵션 입력
            custom_prompt = input("💬 사용자 정의 요약 프롬프트 (엔터 시 기본값): ").strip()
            custom_title = input("📝 사용자 정의 제목 (엔터 시 자동 생성): ").strip()
            
            if process_choice == "1":
                # 첫 번째만 처리 (기존 방식)
                selected_video = videos[0]
                result = await coordinator.process_single_video(
                    url=selected_video['url'],
                    video_info=selected_video,
                    custom_prompt=custom_prompt,
                    custom_title=custom_title
                )
                
            elif process_choice == "2":
                # 모든 영상 처리
                result = await coordinator.process_search_results(
                    search_results=videos,
                    custom_prompt=custom_prompt,
                    custom_title=custom_title,
                    process_all=True
                )
                
            elif process_choice == "3":
                # 특정 영상 선택
                video_choice = input(f"처리할 영상 번호 (1-{len(videos)}): ").strip()
                try:
                    video_index = int(video_choice) - 1
                    if 0 <= video_index < len(videos):
                        selected_video = videos[video_index]
                        result = await coordinator.process_single_video(
                            url=selected_video['url'],
                            video_info=selected_video,
                            custom_prompt=custom_prompt,
                            custom_title=custom_title
                        )
                    else:
                        print("❌ 잘못된 번호입니다. 첫 번째 영상을 처리합니다.")
                        selected_video = videos[0]
                        result = await coordinator.process_single_video(
                            url=selected_video['url'],
                            video_info=selected_video,
                            custom_prompt=custom_prompt,
                            custom_title=custom_title
                        )
                except ValueError:
                    print("❌ 숫자를 입력해주세요. 첫 번째 영상을 처리합니다.")
                    selected_video = videos[0]
                    result = await coordinator.process_single_video(
                        url=selected_video['url'],
                        video_info=selected_video,
                        custom_prompt=custom_prompt,
                        custom_title=custom_title
                    )
            else:
                print("❌ 잘못된 선택입니다. 첫 번째 영상을 처리합니다.")
                selected_video = videos[0]
                result = await coordinator.process_single_video(
                    url=selected_video['url'],
                    video_info=selected_video,
                    custom_prompt=custom_prompt,
                    custom_title=custom_title
                )
        else:
            # 검색 실패 처리
            print(f"\n❌ 검색 실패: {search_result['error']}")
            result = None
            
    elif choice == "2":
        # URL 방식
        target_url = input("🔗 YouTube URL을 입력하세요: ").strip()
        if not target_url:
            target_url = "https://youtu.be/6hSjOCUmjGQ?si=ES5FOYHaFjWtguUl"
        
        video_info = {'url': target_url, 'title': '직접 입력 영상'}
        
        # 처리 실행
        custom_prompt = input("💬 사용자 정의 요약 프롬프트 (엔터 시 기본값): ").strip()
        custom_title = input("📝 사용자 정의 제목 (엔터 시 자동 생성): ").strip()
        
        result = await coordinator.process_single_video(
            url=target_url,
            video_info=video_info,
            custom_prompt=custom_prompt,
            custom_title=custom_title
        )
    else:
        print("❌ 잘못된 선택입니다. 1 또는 2를 입력해주세요.")
        result = None
    
    # 결과 출력
    if result and result['overall_success']:
        print("\n🎊 모든 작업이 성공적으로 완료되었습니다!")
        
        # 여러 결과 처리
        if isinstance(result['final_result'], list):
            # 여러 영상 처리된 경우
            print(f"\n📖 생성된 Notion 페이지들 ({len(result['final_result'])}개):")
            for i, res in enumerate(result['final_result']):
                print(f"🔗 {i+1}. {res['page_url']}")
        else:
            # 단일 영상 처리된 경우
            notion_url = result['final_result']['page_url']
            print(f"\n📖 생성된 Notion 페이지:")
            print(f"🔗 {notion_url}")
    elif result:
        print(f"\n❌ 작업 실패:")
        for error in result['errors']:
            print(f"  • {error}")
    else:
        print("\n❌ 처리할 수 없습니다.")

## 📋 완료!

###  YouTube + Gemini + Notion 통합 시스템 완성!

**🔄 완전한 자동화 파이프라인:**
1. ✅ **YouTube 자막 추출** - 최신 API로 다양한 언어 지원
2. ✅ **Gemini AI 요약** - Google의 최신 AI로 고품질 요약
3. ✅ **Notion 페이지 생성** - 구조화된 페이지로 자동 저장
4. ✅ **사용자 정의** - 프롬프트와 제목 커스터마이징 지원

### 💡 주요 특징:
- 🎯 **원클릭 자동화**: URL 입력만으로 전체 프로세스 완료
- 📊 **풍부한 메타데이터**: 언어, 길이, 토큰 사용량 등 상세 정보
- 📝 **구조화된 페이지**: 요약 + 원문 자막을 깔끔하게 정리
- 🔗 **편리한 접근**: 생성된 Notion 페이지 직접 링크 제공

### 🛠️ 환경 설정 요구사항:
```env
NOTION_TOKEN="your API key"
notion_parent_page="your API key"
GEMINI_API_KEY="your API key"
NOTION_DATABASE_ID="your API key"
```

### 🔗 관련 파일:
- **[youtube_gemini_summary.py](computer:///mnt/user-data/outputs/youtube_gemini_summary.py)** - 단독 스크립트 버전
- **[ai_agent_backend.py](computer:///mnt/user-data/outputs/ai_agent_backend.py)** - 전체 백엔드 시스템

**이제 YouTube 영상을 입력하면 자동으로 요약되어 Notion에 깔끔하게 정리됩니다!** 🚀✨